## Transformer

### Architecture

In [294]:
import torch
import torch.nn as nn
import math
import numpy as np

#### `SelfAttention`

In [295]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * self.heads == self.embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0] # Batch size
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # Shapes before splitting:
        # values:  (N, value_len, embed_size)
        # keys:    (N, key_len  , embed_size)
        # queries: (N, query_len, embed_size)

        # Split the embedding into self.heads different pieces
        values = self.values(values).reshape(N, value_len, self.heads, self.head_dim)
        keys = self.keys(keys).reshape(N, key_len, self.heads, self.head_dim)
        queries = self.queries(query).reshape(N, query_len, self.heads, self.head_dim)
        # Shapes after splitting: 
        # values:  (N, value_len, heads, head_dim)
        # keys:    (N, key_len  , heads, head_dim)
        # queries: (N, query_len, heads, head_dim)

        # Compute the dot product between queries and keys for each head, 
        # and divide by sqrt of head_dim for numerical stability
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]) / math.sqrt(self.head_dim)
        # Shape of energy: (N, heads, query_len, key_len)

        # Apply mask
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        # Shape of mask: (N, 1, 1, key_len)
        # 0 in key_len dimension means that the respective element in energy is set to -1e20
        # Mask will be broadcasted to (N, heads, query_len, key_len) by PyTorch automatically

        # Compute the attention weights for each head using the softmax function
        attention = torch.softmax(energy, dim=-1)
        # Shape of attention: (N, heads, query_len, key_len)

        # Multiply the attention weights with the values for each head and then concatenate
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.embed_size
        )
        # Shape of out: (N, query_len, embed_size)

        out = self.fc_out(out)
        return out

#### `TransformerBlock`: 

`SelfAttention` -> layerNorm -> Feed-Forward -> layerNorm

In [296]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        # Compute self-attention
        attention = self.attention(value, key, query, mask)
        # Shape of attention: (N, query_len, embed_size)

        # Add skip connection, run through normalization and dropout
        x = self.dropout(self.norm1(attention + query))
        # Shape of x: (N, query_len, embed_size)

        # Feed-forward network
        forward = self.feed_forward(x)
        # Shape of forward: (N, query_len, embed_size)

        # Add skip connection, run through normalization and dropout
        out = self.dropout(self.norm2(forward + x))
        # Shape of out: (N, query_len, embed_size)

        return out

#### Encoder: `num_layers` of `TransformerBlock` 

In [297]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size, # Size of the source vocabulary
        num_layers, # Number of TransformerBlocks
        max_length, # Maximum length of the sentence
        embed_size,
        heads,
        forward_expansion,
        dropout,
        device,
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    forward_expansion,
                    dropout,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape

        # Positions is the index of the word in the sentence (0, 1, 2, ..., seq_length)
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        # Add word embeddings and position embeddings
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )
        # Shape of out: (N, seq_length, embed_size)

        # In the Encoder the query, key, value are all the same
        for layer in self.layers:
            out = layer(out, out, out, mask)
            # Shape of out: (N, seq_length, embed_size)

        return out

#### `DecoderBlock`

Masked `SelfAttention` -> layerNorm -> `TransformerBlock`

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.masked_attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, forward_expansion, dropout
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_value, encoder_key, src_mask, trg_mask):
        # Self attention on the target sentence with mask
        attention = self.masked_attention(x, x, x, trg_mask)

        # Add skip connection, run through normalization and dropout
        query = self.dropout(self.norm(attention + x))

        # Transformer block with encoder's output as value and key
        out = self.transformer_block(encoder_value, encoder_key, query, src_mask)
        # Shape of out: (N, query_len, embed_size)
        
        return out

#### Decoder: `num_layers` of `DecoderBlock`

In [301]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size, # Size of the target vocabulary
        num_layers, # Number of DecoderBlocks
        max_length, # Maximum length of the sentence
        embed_size,
        heads,
        forward_expansion,
        dropout,
        device,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(
                    embed_size,
                    heads,
                    forward_expansion,
                    dropout)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape

        # Positions is the index of the word in the sentence (0, 1, 2, ..., seq_length)
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        # Add word embeddings and position embeddings
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        # In the Decoder the key and value are the encoder's output,
        # and the query is the output of the previous DecoderBlock
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out

#### `Transformer`

In [302]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx, # Index of the padding token in the source vocabulary
        trg_pad_idx, # Index of the padding token in the target vocabulary
        num_layers,
        max_length,
        embed_size,
        heads,
        forward_expansion=4,
        dropout=0.0,
        device="cpu",
    ):

        super(Transformer, self).__init__()

        # Initialize the Encoder
        self.encoder = Encoder(
            src_vocab_size, # Size of the source vocabulary
            num_layers, # Number of TransformerBlocks
            max_length, # Maximum length of the sentence
            embed_size,
            heads,
            forward_expansion,
            dropout,
            device,
        )

        # Initialize the Decoder
        self.decoder = Decoder(
            trg_vocab_size, # Size of the target vocabulary
            num_layers, # Number of DecoderBlocks
            max_length, # Maximum length of the sentence
            embed_size,
            heads,
            forward_expansion,
            dropout,
            device,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        # Shape of src: (N, src_len)
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # Shape of src_mask: (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape

        # Create a lower triangular matrix of ones with shape (trg_len, trg_len),
        # then expand it to (N, 1, trg_len, trg_len)
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

### Train the `Transformer`

#### Hyperparameters

In [343]:
# Hyperparameters
num_samples = 5000 # Number of samples in the dataset
max_length = 8 # Maximum length of the sequence including the <SOS> and <EOS> tokens
vocab_size = 99 + 3 # Numbers from 1 to 99 and three indices for padding, start of sequence, and end of sequence
sos_idx = 100 # Start of sequence index
eos_idx = 101 # End of sequence index
pad_idx = 0 # Padding index
num_layers = 2 # Number of Blocks in the Encoder and Decoder
embed_size = 32 # Embedding size for the tokens
heads = 2 # Number of heads in the Multi-Head Attention
forward_expansion = 4
dropout = 0.0
learning_rate = 0.001
batch_size = 256
num_epochs = 20

#### Data Generation

In [344]:
# Function to generate data for the three tasks: copy, reverse, and sort
def generate_data(num_samples, max_length, pad_idx, sos_idx, eos_idx, task):
    src_data = []
    trg_data = []
    
    for _ in range(num_samples):
        seq_length = np.random.randint(1, max_length - 1)  # Length of the random sequence
        sequence = torch.randint(1, 100, (seq_length,))  # Numbers from 1 to 99
        
        # Create the source sequence with <SOS> at the start and <EOS> at the end
        src_sequence = torch.cat([torch.tensor([sos_idx]), sequence, torch.tensor([eos_idx])])
        
        # Pad the source sequence to the maximum length
        padded_src_sequence = torch.cat([src_sequence, torch.full((max_length - len(src_sequence),), pad_idx)])
        
        if task == 'copy':
            # For copying task, target is the same as source
            padded_trg_sequence = padded_src_sequence.clone()
        
        elif task == 'reverse':
            # Reverse the sequence for the target and pad it to the maximum length
            reversed_sequence = sequence.flip(0)
            trg_sequence = torch.cat([torch.tensor([sos_idx]), reversed_sequence, torch.tensor([eos_idx])])
            padded_trg_sequence = torch.cat([trg_sequence, torch.full((max_length - len(trg_sequence),), pad_idx)])
        
        elif task == 'sort':
            # Sort the sequence for the target and pad it to the maximum length
            sorted_sequence = sequence.sort().values
            trg_sequence = torch.cat([torch.tensor([sos_idx]), sorted_sequence, torch.tensor([eos_idx])])
            padded_trg_sequence = torch.cat([trg_sequence, torch.full((max_length - len(trg_sequence),), pad_idx)])
        
        else:
            raise ValueError("Invalid task. Choose from 'copy', 'reverse', or 'sort'.")
        
        src_data.append(padded_src_sequence)
        trg_data.append(padded_trg_sequence)
    
    src_data = torch.stack(src_data)
    trg_data = torch.stack(trg_data)
    
    return src_data, trg_data

# Generate data for the copying task
src_data_copy, trg_data_copy = generate_data(3, max_length, pad_idx, sos_idx, eos_idx, task='copy')

# Generate data for the reversing task
src_data_reverse, trg_data_reverse = generate_data(3, max_length, pad_idx, sos_idx, eos_idx, task='reverse')

# Generate data for the sorting task
src_data_sort, trg_data_sort = generate_data(3, max_length, pad_idx, sos_idx, eos_idx, task='sort')

print("Copy Task - Source Data:")
print(src_data_copy)
print("Copy Task - Target Data:")
print(trg_data_copy)
print()
print("Reverse Task - Source Data:")
print(src_data_reverse)
print("Reverse Task - Target Data:")
print(trg_data_reverse)
print()
print("Sort Task - Source Data:")
print(src_data_sort)
print("Sort Task - Target Data:")
print(trg_data_sort)

Copy Task - Source Data:
tensor([[100,  52,   7,  58,  78,  78, 101,   0],
        [100,  16,  55, 101,   0,   0,   0,   0],
        [100,  24,  35, 101,   0,   0,   0,   0]])
Copy Task - Target Data:
tensor([[100,  52,   7,  58,  78,  78, 101,   0],
        [100,  16,  55, 101,   0,   0,   0,   0],
        [100,  24,  35, 101,   0,   0,   0,   0]])

Reverse Task - Source Data:
tensor([[100,  98,  10,  20,  97,  16,  91, 101],
        [100,  21,  11,  88, 101,   0,   0,   0],
        [100,  43,  15, 101,   0,   0,   0,   0]])
Reverse Task - Target Data:
tensor([[100,  91,  16,  97,  20,  10,  98, 101],
        [100,  88,  11,  21, 101,   0,   0,   0],
        [100,  15,  43, 101,   0,   0,   0,   0]])

Sort Task - Source Data:
tensor([[100,  87,  90,  37,  85,  98,  51, 101],
        [100,  74,  74, 101,   0,   0,   0,   0],
        [100,  38,  15,  11,  64, 101,   0,   0]])
Sort Task - Target Data:
tensor([[100,  37,  51,  85,  87,  90,  98, 101],
        [100,  74,  74, 101,   0,   0

#### Training Loop

In [345]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [349]:
# Generate data for the given task
task = 'reverse'
src_data, trg_data = generate_data(num_samples, max_length, pad_idx, sos_idx, eos_idx, task)

# Create DataLoader
train_dataset = TensorDataset(src_data, trg_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the Transformer model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(
    src_vocab_size=vocab_size,
    trg_vocab_size=vocab_size,
    src_pad_idx=pad_idx,
    trg_pad_idx=pad_idx,
    num_layers=num_layers,
    max_length=max_length,
    embed_size=embed_size,
    heads=heads,
    forward_expansion=forward_expansion,
    dropout=dropout,
    device=device
).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for src, trg in train_loader:
        src = src.to(device)
        trg = trg.to(device)

        # Forward pass
        output = model(src, trg[:, :-1])
        output = output.reshape(-1, output.shape[2])
        trg = trg[:, 1:].reshape(-1)

        # Compute loss
        loss = criterion(output, trg)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}')

print("Training complete.")

Epoch [1/20], Loss: 4.272413802146912
Epoch [2/20], Loss: 4.081902801990509
Epoch [3/20], Loss: 3.96914746761322
Epoch [4/20], Loss: 3.771239864826202
Epoch [5/20], Loss: 3.5348363876342774
Epoch [6/20], Loss: 3.2725308179855346
Epoch [7/20], Loss: 2.8912335872650146
Epoch [8/20], Loss: 2.313313138484955
Epoch [9/20], Loss: 1.64979829788208
Epoch [10/20], Loss: 1.0643664211034776
Epoch [11/20], Loss: 0.6409927934408188
Epoch [12/20], Loss: 0.3919969603419304
Epoch [13/20], Loss: 0.25072157457470895
Epoch [14/20], Loss: 0.1724546119570732
Epoch [15/20], Loss: 0.12661609649658204
Epoch [16/20], Loss: 0.0977422546595335
Epoch [17/20], Loss: 0.07838640697300434
Epoch [18/20], Loss: 0.06462651155889035
Epoch [19/20], Loss: 0.05438430868089199
Epoch [20/20], Loss: 0.04661877769976854
Training complete.


### Evaluating the model

In [359]:
# Example input sequence
input_sequence = [2,7,8,1]

# Prepare the input sequence: Add <SOS> and <EOS> tokens, pad the sequence to the maximum length
input_tensor = torch.tensor([sos_idx] + input_sequence + [eos_idx] + [pad_idx] * (max_length - len(input_sequence) - 2)).unsqueeze(0)  # Add batch dimension

# Move the input tensor to the appropriate device
input_tensor = input_tensor.to(device)
print("Input Tensor:", input_tensor)

Input Tensor: tensor([[100,   2,   7,   8,   1, 101,   0,   0]])


In [360]:
# Function to generate output from the model
def generate_output(model, input_tensor, max_length, pad_idx, sos_idx, eos_idx):
    model.eval()
    with torch.no_grad():
        # Create a target tensor filled with pad_idx
        trg_tensor = torch.full((1, max_length), pad_idx).to(device)
        
        # Set the first token of the target tensor to the start token
        trg_tensor[0, 0] = sos_idx
        
        for i in range(1, max_length):
            # Pass the input and target tensors through the model
            output = model(input_tensor, trg_tensor[:, :i])
            
            # Get the token with the highest probability
            next_token = output.argmax(2)[:, -1]
            
            # Set the next token in the target tensor
            trg_tensor[0, i] = next_token.item()
            
            # Stop if the next token is the end token
            if next_token.item() == eos_idx:
                break
        
        return trg_tensor

# Generate output
output_tensor = generate_output(model, input_tensor, max_length, pad_idx, sos_idx, eos_idx)

print("Output Tensor:", output_tensor)

Output Tensor: tensor([[100,   1,   8,   7,   2, 101,   0,   0]])
